In [124]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [125]:
this_month='1709'
data_path='../data/'+this_month
columns=8

In [126]:
y17 = glob.glob(data_path+'/2017/*.txt')
y16 = glob.glob(data_path+'/2016/*.txt')
y15 = glob.glob(data_path+'/2015/*.txt')
y14 = glob.glob(data_path+'/2014/*.txt')
y14_17=y14+y15+y16+y17
with open(data_path+'/merged.txt', "wb") as outfile:
    for f in y14_17:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==columns:
                    outfile.write(line+'\n')

In [127]:
df=pd.read_table(data_path+'/merged.txt',header=None)
df.columns=['doi','year','citation','issnp','issne','journal','publisher','lic']
df.shape

(10867268, 8)

In [128]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(10867268, 8)

In [129]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [130]:
df['yy']=df.year.str[1:5]
df['mm']=df.year.apply(lambda x: int(x.replace(']','').split(', ')[1]) if len(x.split(', '))>1 else 0)

In [131]:
df['year']=df['yy'].astype(int)

In [132]:
df=df.fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [133]:
s=df.issn.str.split(',',expand=True).stack()
i = s.index.get_level_values(0)
df2 = df.loc[i].copy()
df2["@id"] = s.values

In [134]:
all_issns=set(df.issn.values)

In [135]:
issn_count=df2[['@id','count']].groupby('@id').count().reset_index()

In [136]:
issn_count_dic={}
for index,row in issn_count.iterrows():
    issn_count_dic[row['@id']]=row['count']

In [137]:
issn_pairs_list=[]
main_issns={}
for issn in all_issns:
    if len(issn)>9:
        issn0=issn[:9]; 
        issn1=issn[10:]; 
        if issn1 not in issn_count_dic:
            main_issn=issn0
        if issn0 not in issn_count_dic:
            main_issn=issn1
        if issn0 in issn_count_dic and issn1 in issn_count_dic:
            dif=issn_count_dic[issn1]-issn_count_dic[issn0]
            if dif> 0: 
                main_issn=issn1
            else:
                main_issn=issn0
    else:
        main_issn=issn
    if main_issn in main_issns:
        if main_issns[main_issn]!=issn:
            main_issns[main_issn]+=';'+issn
    else:
        main_issns[main_issn]=issn

In [138]:
pd.DataFrame(main_issns.keys()).to_csv('../store/main_issns.csv',header=False,index=False)
len(main_issns)

42339

In [139]:
issn_to_main={}
for main_issn in main_issns.keys():
    for issn in main_issns[main_issn].split(';'):
        issn_to_main[issn]=main_issn

In [140]:
len(issn_to_main)

45110

In [141]:
df['main_issn']=df.issn.apply(lambda x:issn_to_main[x])

In [142]:
df.to_csv(data_path+'/cleaned.csv',index=False)